In [1]:
import os

In [9]:
type(os.walk("../data/interim/generated"))

generator

In [8]:
for i in os.walk("../data/interim/generated"):
    print(i,end="\n")

('../data/interim/generated', [], ['data_{51}_to_{100}.json', 'data_{101}_to_{150}.json', 'data_{1}_to_{50}.json'])


In [13]:
for i in sorted(os.listdir("/home/jeet/Documents/Jeet/FT_LLM/data/interim/generated")):
    print(i)

001_to_50.json
051_t0_100.json
101_to_150.json


In [15]:
type(os.scandir("/home/jeet/Documents/Jeet/FT_LLM/data/interim/generated"))

posix.ScandirIterator

In [17]:
for i in os.scandir("../data/interim/generated"):
    print(i)

<DirEntry '101_to_150.json'>
<DirEntry '051_t0_100.json'>
<DirEntry '001_to_50.json'>


In [18]:
with os.scandir("../data/interim/generated") as entries:
    for entry in entries:
        print(entry.name)

101_to_150.json
051_t0_100.json
001_to_50.json


In [26]:
path = """../data/interim/generated/"""
for i in sorted(os.listdir(path)):
    print(i)

data_{101}_to_{150}.json
data_{1}_to_{50}.json
data_{51}_to_{100}.json


In [27]:
items = os.listdir(path)
soreted_items = sorted(items)
print(soreted_items)

['data_{101}_to_{150}.json', 'data_{1}_to_{50}.json', 'data_{51}_to_{100}.json']


In [1]:
mylist = ["j","shlf","kdhfsd","jd"]

for i in sorted(mylist,key=len):
    print(i)

j
jd
shlf
kdhfsd


In [13]:
class Person:
    def __init__(self) -> None:
        pass

    def walk(self):
        print("person is walking")
    def talk(self):
        print("person is talking")


class Jeet(Person):
    def __init__(self) -> None:
        # super().__init__()?
        pass

    def walk(self):
        print("jeet is walking")



In [14]:
p = Person()
p.walk()

person is walking


In [15]:
j = Jeet()
j.talk()

person is talking


In [3]:
from transformers import AutoTokenizer


In [4]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
text = "my name is jeet"
decoded_text = tokenizer(text)
print(decoded_text)

{'input_ids': [2577, 1416, 310, 5139, 292], 'attention_mask': [1, 1, 1, 1, 1]}


In [15]:
thisList = ["myname","is","jeet"]
thatlist = [90,345,34]

final_list = thisList.extend(thatlist)
print(final_list)

['myname', 'is', 'jeet', 90, 345, 34]


In [11]:
def extend_list(l1,l2):
    return l1.extend(l2)

final_list = extend_list(thisList,thatlist)
print(final_list)

None


In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
filename = "../dataset.jsonl"

In [3]:
import pandas
instruction_dataset_df = pandas.read_json(filename,lines=True)
examples = instruction_dataset_df.to_dict() 

In [16]:
with open("converted.txt","w") as file:
    file.write(str(examples['input'][0]))

In [8]:
print(examples['input'][0])
print(examples['output'][0])

Are you ready to build your dream home in one of Melbourne's most sought-after communities? This EAST facing 448 square meter vacant land at 15 Caspian Street, Bonnie Brook, within the prestigious Woodlea Estate, presents an incredible opportunity for savvy buyers looking for the perfect canvas to design and build their ideal home.

Key Features:

Generous Land Size: With a spacious 448 square meters to work with, there's ample room to create a stunning residence that suits your lifestyle and preferences.

Woodlea Estate: Woodlea is renowned for its community-oriented living, offering a vibrant and family-friendly atmosphere. You'll have access to parks, playgrounds, schools, shopping, and more, making it an ideal place to call home.
Woodlea has been carefully designed to connect residents with everything they need. Every home is within 200m of a park, with every park and neighbourhood able to be accessed via various walkable routes. With work and education facilities on the doorstep a

In [10]:
text = examples['input'][0] + str(examples['output'][0])

In [11]:
text

"Are you ready to build your dream home in one of Melbourne's most sought-after communities? This EAST facing 448 square meter vacant land at 15 Caspian Street, Bonnie Brook, within the prestigious Woodlea Estate, presents an incredible opportunity for savvy buyers looking for the perfect canvas to design and build their ideal home.\n\nKey Features:\n\nGenerous Land Size: With a spacious 448 square meters to work with, there's ample room to create a stunning residence that suits your lifestyle and preferences.\n\nWoodlea Estate: Woodlea is renowned for its community-oriented living, offering a vibrant and family-friendly atmosphere. You'll have access to parks, playgrounds, schools, shopping, and more, making it an ideal place to call home.\nWoodlea has been carefully designed to connect residents with everything they need. Every home is within 200m of a park, with every park and neighbourhood able to be accessed via various walkable routes. With work and education facilities on the do

In [15]:
print(len(examples["input"]))

150


In [20]:
def tokenize_datapoint(finetuning_dataset_dict):
    tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
    description = finetuning_dataset_dict["input"][0]
    output = finetuning_dataset_dict["output"][0]
    text = description + str(output)

    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(
        text = text,
        return_tensors = "np",
        padding = True
    )

    max_length = min(tokenized_inputs["input_ids"].shape[1], 2048)

    tokenized_inputs = tokenizer(
        text = text,
        return_tensors = "np",
        truncation = True,
        max_length = max_length
    )
    return tokenized_inputs

In [21]:
from datasets import load_dataset
path = "../dataset.jsonl"
finetuning_dataset_hf = load_dataset(
    "json",
    data_files=path,
    split="train"
)

tokenized_dataset = finetuning_dataset_hf.map(
    tokenize_datapoint,
    batched=True,
    batch_size=1,
    drop_last_batch=True
)
tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_dataset["input_ids"])
splitted_dataset = tokenized_dataset.train_test_split(test_size=0.1, shuffle=True, seed=123)

# return splitted_dataset["train"], splitted_dataset["test"]

Map: 100%|██████████| 150/150 [00:43<00:00,  3.43 examples/s]


In [22]:
print(splitted_dataset["test"])

Dataset({
    features: ['id', 'input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 15
})


In [24]:
import create_token
 

ModuleNotFoundError: No module named 'create_token'

In [25]:
import data_generate

ModuleNotFoundError: No module named 'data_generate'

In [28]:
from FT_LLM import create_token

ModuleNotFoundError: No module named 'FT_LLM'

In [29]:
from data.final import finetuning_dataset_hf

ModuleNotFoundError: No module named 'data'

In [30]:
from create_token import the_final_dataset

ModuleNotFoundError: No module named 'create_token'

In [31]:
import templates

In [35]:
import create_token

ModuleNotFoundError: No module named 'create_token'

In [38]:
import sys
sys.path.append("/home/jeet/Jeet/FT_LLM/")

In [43]:
from create_token import main

In [49]:
print(main)

<function main at 0x7fc56cf5a0e0>


In [56]:
print(create_token.thisis)

AttributeError: module 'create_token' has no attribute 'thisis'

In [57]:
(Dataset({
    features: ['id', 'input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 135
}), Dataset({
    features: ['id', 'input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 15
}))

NameError: name 'Dataset' is not defined